# Prueba Técnica

Nuestra empresa conecta proveedores de productos de viajes y distribuidores de viajes. 
Objetivo: limpiar y preparar un conjunto de datos antes de cargarlo en un data warehouse para su análisis. 
Dataset: contiene información como reservas afectadas, solicitantes, aprobadores de estas solicitudes, precios, moneda, motivo de solicitud, estado y datos del cliente. 
Requisitos: 
- No debe haber más de dos peticiones para la misma reserva y misma cantidad. 
- Todas las peticiones deben tener a una persona en Authorized by. 
- Todos los correos deben tener el formato correcto. 
- Todas las cantidades deben cambiarse a euros y hay que borrar la columna Amount_comges_in_EUR 
- La columna “reason” es obligatoria, la “reason 2”, recomendada. 
- Hacer una breve visualización de los datos para entenderlos mejor.

In [35]:
pip install pandas

In [36]:
import pandas as pd

In [37]:
# Importar librerías necesarias

# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np


# Visualización
# ------------------------------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns

# Evaluar linealidad de las relaciones entre las variables
# y la distribución de las variables
# ------------------------------------------------------------------------------
#import scipy.stats as stats
import scipy.stats as st
import scipy.stats as stats
from scipy.stats import shapiro, poisson, chisquare, expon, kstest

# Configuraciones
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames

In [40]:
# Carga del Data Frame

df = pd.read_excel("requests.xlsx", index_col = 0)
df.head(3)

KeyboardInterrupt: 

In [ ]:
# Realizamos una exploración inicial de los datos para identificar posibles problemas, como valores nulos, atípicos o datos faltantes en las columnas relevantes
# Solicitamos visión general del DF
display(df.head(3))
display(df.tail(3))
display(df.sample(3))

,Request date,Requested by,Authorized by,Department,Currency,Amount,Reason,Reason 2,Status,CustomerShortname,CustomerRegion,Amount COMGES in EUR
Booking,,,,,,,,,,,,
100/1000000,2024-01-02 00:00:00,user94@hotelbeds.com,approver21@hotelbeds.com,Sales,CNY,74.82,BOOKING_OPERATIONAL_ISSUE,RESERVATION DISCREPANCIES,Applied,CLIENT82,Region 4,9.61
100/1000001,2024-01-02 00:00:00,user94@hotelbeds.com,approver21@hotelbeds.com,Sales,CNY,424.37,BOOKING_OPERATIONAL_ISSUE,RESERVATION DISCREPANCIES,Applied,CLIENT141,Region 1,54.52
100/1000002,2024-01-02 00:00:00,user94@hotelbeds.com,approver21@hotelbeds.com,Sales,CNY,104.00,BOOKING_TECHNICAL_ISSUE,PRICE DISCREPANCY ACROSS BOOKING PROCESS,Applied,CLIENT141,Region 1,13.36


,Request date,Requested by,Authorized by,Department,Currency,Amount,Reason,Reason 2,Status,CustomerShortname,CustomerRegion,Amount COMGES in EUR
Booking,,,,,,,,,,,,
100/1239397,2024-07-19 00:00:00,user7@hotelbeds.com,approver40@hotelbeds.com,Sales,CAD,345.63,BOOKING_OPERATIONAL_ISSUE,RESERVATION DISCREPANCIES,Requested,CLIENT283,Region 3,235.73
100/1239398,2024-07-19 00:00:00,user7@hotelbeds.com,approver40@hotelbeds.com,Sales,CAD,78.75,BOOKING_OPERATIONAL_ISSUE,RESERVATION DISCREPANCIES,Requested,CLIENT283,Region 3,53.71
100/1239399,2024-07-22 00:00:00,user44@hotelbeds.com,approver42@hotelbeds.com,Sales,IDR,7003421.00,OTHERS,PRICE MATCH,Requested,CLIENT120,Region 1,399.28


,Request date,Requested by,Authorized by,Department,Currency,Amount,Reason,Reason 2,Status,CustomerShortname,CustomerRegion,Amount COMGES in EUR
Booking,,,,,,,,,,,,
100/1109904,2024-04-22 00:00:00,user23@hotelbeds.com,approver25@hotelbeds.com,Sales,USD,11.50,OTHERS,SPECIFIC CUSTOMER AGREEMENTS,Applied,CLIENT1,Region 1,10.74
100/1113906,2024-04-22 00:00:00,user23@hotelbeds.com,approver25@hotelbeds.com,Sales,USD,1.31,OTHERS,SPECIFIC CUSTOMER AGREEMENTS,Applied,CLIENT3,Region 3,1.22
100/1144753,2024-05-30 00:00:00,user23@hotelbeds.com,approver25@hotelbeds.com,Sales,USD,3.84,OTHERS,SPECIFIC CUSTOMER AGREEMENTS,Applied,CLIENT1,Region 1,3.58


In [ ]:
df.shape

(239400, 12)

In [ ]:
print(f"El número de filas es {df.shape[0]} y el número de columnas es {df.shape[1]}")

El número de filas es 239400 y el número de columnas es 12


In [41]:
# Los nombres de las columnas son correctos y claros, pero como buena práctica de Python y Pandas, cambiamos los espacios por guiones bajos
nuevas_columnas = {col: col.replace(' ','_') for col in df.columns}
nuevas_columnas

{'Request date': 'Request_date',
 'Requested by': 'Requested_by',
 'Authorized by': 'Authorized_by',
 'Department': 'Department',
 'Currency': 'Currency',
 'Amount': 'Amount',
 'Reason': 'Reason',
 'Reason 2': 'Reason_2',
 'Status': 'Status',
 'CustomerShortname': 'CustomerShortname',
 'CustomerRegion': 'CustomerRegion',
 'Amount COMGES in EUR': 'Amount_COMGES_in_EUR'}

In [42]:
df.rename(columns = nuevas_columnas, inplace=True)
df.columns

Index(['Request_date', 'Requested_by', 'Authorized_by', 'Department',
       'Currency', 'Amount', 'Reason', 'Reason_2', 'Status',
       'CustomerShortname', 'CustomerRegion', 'Amount_COMGES_in_EUR'],
      dtype='object')

In [44]:
# Solicitamos información general del DF
df.info() # observamos que hay nulos en algunas columnas y que los datos son todos de tipo texto o numéricos decimales.

<class 'pandas.core.frame.DataFrame'>
Index: 239400 entries, 100/1000000 to 100/1239399
Data columns (total 12 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   Request_date          239400 non-null  object 
 1   Requested_by          239400 non-null  object 
 2   Authorized_by         239398 non-null  object 
 3   Department            239400 non-null  object 
 4   Currency              239400 non-null  object 
 5   Amount                239400 non-null  float64
 6   Reason                239397 non-null  object 
 7   Reason_2              239316 non-null  object 
 8   Status                239400 non-null  object 
 9   CustomerShortname     239400 non-null  object 
 10  CustomerRegion        239400 non-null  object 
 11  Amount_COMGES_in_EUR  239392 non-null  float64
dtypes: float64(2), object(10)
memory usage: 23.7+ MB


In [45]:
# Utilizamos un .describe() para obtener una visión general de las principales estadísticas descriptivas
# Transponemos los resultados con .T para tener una mejor lectura
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Amount,239400.00,1408.66,184094.65,-219.90,1.89,2.73,4.58,38000000.00
Amount_COMGES_in_EUR,239392.00,12.98,99.62,-219.90,1.77,2.56,4.28,18356.96


In [46]:
# Calculamos cuántos valores nulos hay en las columnas que los tienen
print("Valores nulos en el df:")
df.isnull().sum()

Valores nulos en el df:


Request_date             0
Requested_by             0
Authorized_by            2
Department               0
Currency                 0
Amount                   0
Reason                   3
Reason_2                84
Status                   0
CustomerShortname        0
CustomerRegion           0
Amount_COMGES_in_EUR     8
dtype: int64

In [47]:
# Utilizamos .duplicated() para identificar filas duplicadas en el DF
print("Serie booleana de filas duplicadas (considerando todas las columnas):")
df.duplicated()

Serie booleana de filas duplicadas (considerando todas las columnas):


Booking
100/1000000    False
100/1000001    False
100/1000002    False
100/1000003    False
100/1000004    False
               ...  
100/1239395    False
100/1239396    False
100/1239397    False
100/1239398    False
100/1239399    False
Length: 239400, dtype: bool

In [48]:
# Obtenemos el conteo de filas duplicadas
print("\nNúmero total de filas duplicadas en el DataFrame:")
df.duplicated().sum()


Número total de filas duplicadas en el DataFrame:


168516